#  preprocessing

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline
from keras.utils.vis_utils import plot_model,model_to_dot
from sklearn.metrics import mean_squared_error
from math import sqrt

# Importing the Keras libraries and packages
from keras.models import Sequential,load_model
from keras.layers import Dense
from keras.layers import GRU,LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import TimeSeriesSplit
from ipywidgets import interactive

In [ ]:
!pip install scikit-optimize

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

timeseriessplit=TimeSeriesSplit(n_splits=5)

In [ ]:
df=pd.read_csv("../input/m5-forecasting-accuracy/sales_train_evaluation.csv")

In [ ]:
df.head()

In [ ]:
df["type"]=df["store_id"]+"_"+df["dept_id"]

In [ ]:
df.head()

In [ ]:
store_with_type=df.groupby("type").sum().T

In [ ]:
store_with_type.head()

In [ ]:
calendar=pd.read_csv("../input/m5-forecasting-accuracy/calendar.csv",parse_dates=True)
calendar['date'] = pd.to_datetime(calendar['date'])
calendar.set_index(calendar["d"],inplace=True)
calendar.drop(labels="d",axis=1,inplace=True)

In [ ]:
combined=pd.concat([store_with_type,calendar],axis=1,join="inner")

In [ ]:
combined.head()

In [ ]:
combined.drop(["wm_yr_wk","wday",'event_name_1', 'event_type_1','event_name_2', 'event_type_2'],axis=1,inplace=True)

In [ ]:
combined.drop(['weekday', 'month', 'year', 'snap_CA', 'snap_TX', 'snap_WI'],axis=1,inplace=True)

In [ ]:
combined.set_index(combined["date"],inplace=True)
combined.drop("date",axis=1,inplace=True)

In [ ]:
days=30

train=combined.iloc[:-days]
test=combined.iloc[-days:]

print(train.shape)
print(test.shape)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()
scaled=scaler.fit_transform(train)

timesteps=30

generator=TimeseriesGenerator(scaled,scaled,batch_size=3049, length=timesteps)

In [ ]:
X_train,y_train=generator[0]

X_train.shape

In [ ]:
y_train.shape

# GRU Model creation**

In [ ]:
def GRU_model(optimizer="adam",layer_1_units=300,layer_2_units=300,layer_3_units=300,dropout=0.19,activation='tanh'):
    model=Sequential()
    
    #layer_1_units=40
    model.add(GRU(activation=activation,units = layer_1_units,dropout=dropout, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
    #model.add(Dropout(dropout))
    # Adding a second Grulayer and some Dropout regularisation
    #layer_2_units=neurons
    model.add(GRU(activation=activation,units = layer_2_units,return_sequences=True))
    model.add(Dropout(dropout))

    #layer_3_units=neurons
    model.add(GRU(activation=activation,dropout=dropout,units = layer_3_units))
    model.add(Dropout(dropout))
    
    model.add(Dense(units = train.shape[1]))
    
    model.compile(optimizer = optimizer, loss = 'mean_squared_error',metrics=["mean_absolute_error"])
    
    return model
    

model = KerasRegressor(build_fn=GRU_model,verbose=1)

In [ ]:
search_space = {"layer_1_units": Integer(50,400),
        "layer_2_units": Integer(50,400),
        "layer_3_units": Integer(50,400), 
        "dropout": Real(0.1,0.8),
        "activation": Categorical(['relu', 'tanh']),
        "batch_size": Integer(2,256),
        "epochs": Integer(5,100)}

In [ ]:
bayes = BayesSearchCV(estimator=model, search_spaces=search_space, 
                      n_jobs=1, cv=timeseriessplit,verbose=1,n_iter=30,return_train_score=True)

In [ ]:
bayes_result=bayes.fit(X_train,y_train)

In [ ]:
bayes_result.best_params_

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(bayes_result.best_estimator_.model.history.history["loss"],label="MSE")
plt.plot(bayes_result.best_estimator_.model.history.history["mean_absolute_error"],label="MAE")
plt.legend()

In [ ]:
pd.DataFrame(bayes_result.cv_results_)

In [ ]:
pd.DataFrame(bayes_result.cv_results_).to_csv("bayes_results.csv")

In [ ]:
inputs=train[-days:]
inputs=scaler.transform(inputs)

test_predictions=[]

current_batch=inputs.reshape((1,days,train.shape[1]))

for i in range(days):
    current_pred=gru_model.predict(current_batch)[0]
    
    test_predictions.append(current_pred)
    
    current_batch=np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
    

true_predictions = scaler.inverse_transform(test_predictions)
true_predictions = pd.DataFrame(data=true_predictions,columns=combined.columns)

true_predictions.set_index(test.index,inplace=True)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive,fixed,interact_manual

def compare_plot(a):
  test.iloc[:,a].plot(figsize=(20,8))
  true_predictions.iloc[:,a].plot(figsize=(20,8),label="prediction"+true_predictions.columns[a],linestyle='dashed')
  plt.legend(loc="upper left")
interact(compare_plot,a=(0,train.shape[1]-12,1))
plt.close()

# VAR Model|

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.metrics import mean_squared_error
from math import sqrt
from statsmodels.tsa.stattools import adfuller
import seaborn as sns

In [ ]:
train_differenced = train.diff().dropna()

var_model=VAR(train_differenced)

In [ ]:
res = var_model.select_order(12)
res.summary()

In [ ]:
predictions=var_model_fit.forecast(var_model_fit.y, steps=30)

var_pred=pd.DataFrame(data=predictions,columns=test.columns,index=test.index)


In [ ]:
def invert_transformation(df_train, df_forecast):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc =pd.DataFrame()
    columns = df_train.columns
    for col in columns:
      df_fc[str(col)] = df_train[col].iloc[-1] + df_forecast[str(col)].cumsum()
    return df_fc

df_results = invert_transformation(train,var_pred)

In [ ]:
from ipywidgets import interact, interactive,fixed,interact_manual
import ipywidgets as widgets

def compare_plot_1(a):
  test.iloc[:,a].plot(figsize=(20,8))
  df_results.iloc[:,a].plot(figsize=(20,8),label="VAR "+df_results.columns[a],linestyle='dashed' )
  true_predictions.iloc[:,a].plot(figsize=(20,8),label="GRU "+true_predictions.columns[a],linestyle='dashed')
  plt.legend(loc="upper left", prop={'size': 12})
interact(compare_plot_1,a=(0,df_results.shape[1]-1,1))

# # Model evaluation

In [ ]:
import numpy as np

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

#model evaluation function using 2 dataframes as inputs 
#df1=true values
#df2=predictions 
def MAPE(df1,df2):
  MAPE_values=[]
  series=[]
  for i in range(df1.shape[1]):
    MAPE_values.append(mean_absolute_percentage_error(df1.iloc[:,i],df2.iloc[:,i]))
    series.append(df1.columns[i])
    d = {'series':series,'MAPE':MAPE_values}

  MAPE=pd.DataFrame(MAPE_values,index=df1.columns,columns=["MAPE"])
  return MAPE

In [ ]:
gru_error=MAPE(test,true_predictions)

gru_error["department"]=gru_error.index.str[5:]

In [ ]:
var_error=MAPE(test,df_results)

var_error["department"]=var_error.index.str[5:]

In [ ]:
plt.figure(figsize=(10,5))
gru_error.groupby("department").mean()["MAPE"].plot(label="GRU",color="r")
var_error.groupby("department").mean()["MAPE"].plot(label="VAR",color="k")
plt.legend()
plt.ylabel("MAPE")
plt.xlabel("Department")
plt.savefig("Department_MAPE",bbox_inches="tight")